In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/Kannada-MNIST/Dig-MNIST.csv
/kaggle/input/Kannada-MNIST/train.csv
/kaggle/input/Kannada-MNIST/sample_submission.csv
/kaggle/input/Kannada-MNIST/test.csv


In [2]:
import random

In [3]:
from keras.models import Sequential
from keras import layers
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
# from keras.callbacks import TensorBoard
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [5]:
import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()

In [6]:
train_df = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test_df = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
val_df = pd.read_csv('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv')

In [7]:
train_df.tail()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
59995,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59999,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
val_df.tail()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
10235,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10236,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10237,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10238,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10239,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
test_df.tail()

,id,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
4995,4995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,4996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,4997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,4998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4999,4999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
train_df = pd.concat([train_df, val_df], axis = 0)

In [11]:
Y_train = train_df['label']
Y_id = test_df['id']

In [12]:
train_df.drop('label', axis = 1, inplace = True)
test_df.drop('id', axis = 1, inplace = True)

In [13]:
# X_train = train_df.to_numpy()
# X_test = test_df.to_numpy()

In [14]:
print(train_df.shape)
print(test_df.shape)

(70240, 784)
(5000, 784)


In [15]:
X_train = train_df.values.reshape(train_df.shape[0], 28, 28, 1)
X_test = test_df.values.reshape(test_df.shape[0], 28, 28, 1)

In [16]:
Y_train = np.asarray(Y_train)
print(Y_train.shape)

(70240,)


In [17]:
print(X_train.shape)
print(X_test.shape)

(70240, 28, 28, 1)
(5000, 28, 28, 1)


In [18]:
def plot_digit(dataset, samples, labels):
#     Generate a random set of samples and plot them
#     random_samples = []
    for pts in range(samples):
        r_n = random.randint(0,dataset.shape[0])
#         random_samples  = [23,43,445,44,34]
        img = dataset[r_n].reshape(28,28)
        plt.imshow(img, cmap = 'gray')
        plt.title(labels[r_n])
        plt.show()
        

In [19]:
# plot_digit(X_train, 5, Y_train)

In [20]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [21]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, stratify = Y_train, random_state = 31, shuffle = True, test_size = 0.2)

In [22]:
train_datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.3,
                                   height_shift_range=0.3, brightness_range=None, shear_range=0.3,
                                   zoom_range=0.3, horizontal_flip=False,
                                   vertical_flip=False, rescale=1/255.,
                                   data_format='channels_last', validation_split=0.0,
                                   interpolation_order=1, dtype='float32')

In [23]:
X_val = X_val / 255.                    # Scaling validation Data

In [24]:
X_test = X_test / 255.

In [25]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(56192, 28, 28, 1)
(14048, 28, 28, 1)
(56192, 10)
(14048, 10)


In [26]:
train_datagen.fit(X_train)

In [27]:
model = Sequential()

model.add(layers.Conv2D(128, 3, input_shape = (28,28,1), padding = 'same', activation = None))
model.add(layers.LeakyReLU(alpha = 0.3))
model.add(layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.2))

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(layers.Conv2D(64, 3, padding = 'same', activation = None))
model.add(layers.LeakyReLU(alpha = 0.3))
model.add(layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(layers.Conv2D(64, 3, padding = 'same', activation = None))
model.add(layers.LeakyReLU(alpha = 0.3))
model.add(layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(layers.Conv2D(64, 3, padding = 'same', activation = None))
model.add(layers.LeakyReLU(alpha = 0.3))
model.add(layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(layers.Conv2D(32, 3, padding = 'same', activation = None))
model.add(layers.LeakyReLU(alpha = 0.3))
model.add(layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))

model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.LeakyReLU(alpha = 0.3))
model.add(layers.Dense(10, activation = 'softmax'))



In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 128)       1280      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 128)       112       
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 64)       

In [29]:
opt1 = optimizers.Adam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999)
opt2 = optimizers.Nadam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999)

In [30]:
filepath1 = 'kannada_cnn_model1.hdf5'
filepath2 = 'kannada_cnn_model2.hdf5'
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
callback_l = [reducelr, checkpoint1]
callback_l2 = [reducelr, checkpoint2]

In [31]:
model.compile(optimizer = opt1, loss = 'categorical_crossentropy', metrics = ['acc'])

In [32]:
epochs = 50
batch_size = 128

In [33]:
# print(Y_train[1])

In [34]:
history = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size = batch_size ),
                              epochs = epochs, validation_data = (X_val, Y_val), verbose = 1,
                              steps_per_epoch=X_train.shape[0] // batch_size, callbacks = callback_l)

Epoch 1/50
439/439 [==============================] - 37s 85ms/step - loss: 1.5450 - acc: 0.4517 - val_loss: 2.6321 - val_acc: 0.1012

Epoch 00001: val_loss improved from inf to 2.63214, saving model to kannada_cnn_model1.hdf5
Epoch 2/50
439/439 [==============================] - 30s 69ms/step - loss: 0.8426 - acc: 0.7119 - val_loss: 0.4666 - val_acc: 0.8724

Epoch 00002: val_loss improved from 2.63214 to 0.46660, saving model to kannada_cnn_model1.hdf5
Epoch 3/50
439/439 [==============================] - 32s 73ms/step - loss: 0.6462 - acc: 0.7851 - val_loss: 0.3252 - val_acc: 0.9074

Epoch 00003: val_loss improved from 0.46660 to 0.32518, saving model to kannada_cnn_model1.hdf5
Epoch 4/50
439/439 [==============================] - 31s 70ms/step - loss: 0.5451 - acc: 0.8217 - val_loss: 0.2664 - val_acc: 0.9231

Epoch 00004: val_loss improved from 0.32518 to 0.26637, saving model to kannada_cnn_model1.hdf5
Epoch 5/50
439/439 [==============================] - 30s 69ms/step - loss: 0.48

In [35]:
# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(acc) + 1)

# plt.plot(epochs, acc, 'g', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()
# plt.figure()

# plt.plot(epochs, loss, 'g', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()
# plt.show()

In [36]:
# model.compile(optimizer = opt2, loss = 'categorical_crossentropy', metrics = ['acc'])

In [37]:
# history = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size = batch_size ),
#                               epochs = 50, validation_data = (X_val, Y_val), verbose = 1,
#                               steps_per_epoch=X_train.shape[0] // batch_size, callbacks = callback_l2)

In [38]:
# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(acc) + 1)

# plt.plot(epochs, acc, 'g', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()
# plt.figure()

# plt.plot(epochs, loss, 'g', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()
# plt.show()

In [39]:
best_model = Sequential()

In [40]:
best_model = load_model(filepath1)

In [41]:
y_pred = best_model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

In [42]:
sample_sub = pd.read_csv('/kaggle/input/Kannada-MNIST/sample_submission.csv')
sample_sub['label'] = y_pred
sample_sub.to_csv('submission.csv', index = False)

In [43]:
sample_sub.tail()

,id,label
4995,4995,1
4996,4996,0
4997,4997,1
4998,4998,6
4999,4999,3
